In [1]:
import pickle
import sys
from pathlib import Path

sys.path.insert(0, "/Users/aldcroft/git/ska_sun")
sys.path.insert(0, "/Users/aldcroft/git/ska_helpers")

import astropy.units as u
import numpy as np
import ska_sun
from cxotime import CxoTime
from Quaternion import Quat
from ska_helpers.utils import random_radec_in_cone
from tqdm import tqdm

import find_attitude as fa
from find_attitude.tests import test_find_attitude

In [2]:
np.random.seed(0)

In [3]:
def get_random_attitude(constraints: fa.Constraints) -> Quat:
    """
    Generates a random attitude based on the given constraints.

    Parameters
    ----------
    constraints : fa.Constraints
        The constraints for generating the attitude.

    Returns
    -------
    Quat
        Randomly generated attitude.

    """
    date = constraints.date

    # Back off from constraint off_nom_roll_max by 0.5 degrees to avoid randomly
    # generating an attitude that is too close to the constraint.
    onrm = max(0.0, constraints.off_nom_roll_max - 0.5)
    off_nom_roll = np.random.uniform(-onrm, onrm)

    if constraints.att is not None:
        att0 = Quat(constraints.att)
        # Back off from constraint att_radius by 0.5 degrees (same as off_nom_roll).
        att_radius = max(0.0, constraints.att_radius - 0.5)
        ra, dec = random_radec_in_cone(att0.ra, att0.dec, att_radius)
        roll0 = ska_sun.nominal_roll(ra, dec, time=date)
        roll = roll0 + off_nom_roll
        att = Quat([ra, dec, roll])

    elif constraints.normal_sun:
        normal_sun_radius = max(0.0, constraints.normal_sun_radius - 0.5)
        d_pitch = np.random.uniform(normal_sun_radius, normal_sun_radius)
        pitch = constraints.normal_sun_pitch + d_pitch
        yaw = np.random.uniform(0, 360)
        att = ska_sun.get_att_for_sun_pitch_yaw(
            pitch, yaw, time=date, off_nom_roll=off_nom_roll
        )

    else:
        ra = np.random.uniform(0, 360)
        dec = np.rad2deg(np.arccos(np.random.uniform(-1, 1))) - 90
        roll = ska_sun.nominal_roll(ra, dec, time=date) + off_nom_roll
        att = Quat([ra, dec, roll])

    return att

In [4]:
def test_random_all_sky(
    sigma_1axis=0.4,
    sigma_mag=0.2,
    brightest=True,
    date="2025:001",
    constraints=None,
    min_stars=4,
    max_stars=8,
):
    if constraints is None:
        constraints = fa.Constraints(off_nom_roll_max=20, date=date, normal_sun=False)

    n_stars = np.random.randint(min_stars, max_stars + 1)

    while True:
        att = get_random_attitude(constraints)
        stars = test_find_attitude.get_stars(
            att.ra,
            att.dec,
            att.roll,
            sigma_1axis=sigma_1axis,
            sigma_mag=sigma_mag,
            brightest=brightest,
            date=date,
        )
        if len(stars) >= n_stars:
            break

    stars = stars[:n_stars]
    solutions = fa.find_attitude_solutions(
        stars, constraints=constraints, log_level="WARNING", sherpa_log_level="WARNING"
    )

    for solution in solutions:
        solution["att"] = att

    try:
        test_find_attitude.check_output(solutions, stars, att.ra, att.dec, att.roll)
    except Exception:
        failed_test = {
            "stars": stars,
            "solutions": solutions,
            "att": att,
            "constraints": constraints,
        }
        now = CxoTime.now().isot
        fn = Path(f"failed_test_{now}.pkl")
        print(f"Saving failed test to {fn.absolute()}")
        with open(fn, "wb") as fh:
            pickle.dump(failed_test, fh)

In [5]:
# Estimated OBC attitude at 2024:036:03:03:50.481 during NSM recovery
date_nsm = "2024:036:03:03:50.481"
q_att_nsm = Quat([-0.061655608, -0.054312481, 0.920848154, 0.381165866])

In [6]:
n_test = 100

### Test random all-sky with attitude constraint and three stars

In [7]:
for _ in tqdm(range(n_test)):
    # Get a random sun-pointed attitude anywhere on the sky for a time in 2024 or 2025
    constraints_all_sky = fa.Constraints(
        off_nom_roll_max=0.0,
        date=CxoTime("2024:001") + np.random.uniform(0, 2) * u.yr,
    )
    att_est = get_random_attitude(constraints_all_sky)

    # Now run find attitude test with this constraint where the test attitude will be
    # randomized consistent with the constraints. This selects stars at an attitude
    # which is randomly displaced from the estimated attitude.
    constraints = fa.Constraints(
        off_nom_roll_max=1.0,
        date=constraints_all_sky.date,
        att=att_est,
        att_radius=4.0,
    )
    test_random_all_sky(constraints=constraints, min_stars=3, max_stars=3)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:23<00:00,  4.32it/s]


### Random attitude on sky with no constraints

- This uses between 4 to 8 stars (randomly selected) for each attitude.
- Attitude is random except `off_nominal_roll < 1.0`.

In [8]:
for _ in tqdm(range(n_test)):
    test_random_all_sky(constraints=None)

100%|██████████| 100/100 [02:26<00:00,  1.46s/it]


### Random attitude constraint to pitch near 160 degrees

In [9]:
for _ in tqdm(range(n_test)):
    constraints = fa.Constraints(
        off_nom_roll_max=1.0,
        date=CxoTime("2024:001") + np.random.uniform(0, 2) * u.yr,
        normal_sun=True,
        normal_sun_pitch=160,
    )
    test_random_all_sky(constraints=constraints, min_stars=3, max_stars=3)

100%|██████████| 100/100 [00:25<00:00,  3.98it/s]


In [10]:
# sols = fa.find_attitude_solutions(
#     stars_manual, constraints=constraints_manual, tolerance=3.5
# )
# print(f"Found {len(sols)} solutions")
# sols

### Debug a failed solution

In [11]:
# fn = "failed_test_2024-08-05T11:03:01.043.pkl"
# fn = "failed_test_2024-08-05T17:30:22.539.pkl"
# fn = "failed_test_2024-08-05T20:29:57.752.pkl"
# if Path(fn).exists():
#     fail = pickle.load(open(fn, "rb"))
#     sols = fa.find_attitude_solutions(
#         fail["stars"],
#         constraints=fail["constraints"],
#         log_level="DEBUG",
#         sherpa_log_level="INFO",
#     )